In [8]:
def authenticating(credential):
    logging('\n\nfunction>>>>>authenticating')
    
    auth = tweepy.OAuthHandler(credential["api_key"], credential["api_secret"])
    auth.set_access_token(credential["access_token"], credential["access_token_secret"])
    
    api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    return api

In [56]:
def print_and_retweet_tweet(api,tweet):
    logging('\n\nfunction>>>>>print_and_retweet_tweet')
    
    try: 
        
        logging('INFORMACOES:------------------------------\n\n\n\n')
        logging('tweet object'+str(tweet))
        logging('tweet ID: '+str(tweet.id))
        logging('tweet: '+tweet.text)
        logging('user: '+ str(tweet.user.screen_name))
        logging('created_at: '+str(tweet.created_at))
        logging('place: '+str(tweet.place))
        logging('lang: ' + str(tweet.lang))
        logging('---------------------------------------------\n\n\n\n')
        logging('retweeting now::::::::')
    
        api.retweet(tweet.id)
        
        logging(':::::::: retweeted ')

    except tweepy.TweepError as e: 
        if e.api_code == 327:
            logging('You have already retweeted this Tweet')
            pass
    except tweepy.RateLimitError as e:
        logging('Excedeu limite por tempo?')
        logging('erro: '+str(e))
        logging('DORMINDO POR QUINZE MINUTOS')
        time.sleep(60 * 15)  # we supposedly saw a rate limit that is ignored after 15 min ??? so we should wait 15 min to retry 

In [57]:
def append_tweet_infos_to_lists(tweet, dict_tweets_info):
    logging('\n\nfunction>>>>>append_tweet_infos_to_lists')
    
    # ATTRIBUTES to be transcripted to CSV  - STOLEN DATA !!!!!!!!!!!!!!
    # created_at
    # place
    # user
    # id_str
    # source_url
    # text
    # lang

    # tomar a decisão de dar um append no arquivo atual ou criar um novo toda vez q entrar nessa função
    # o arquivo vai ser um csv

    #dict_tweets_info['created_at'].append(tweet.created_at)
    #dict_tweets_info['place'].append(tweet.place)
    #dict_tweets_info['user'].append(tweet.user.)
    dict_tweets_info['id_str'].append(tweet.id_str)
    #dict_tweets_info['source_url'].append(tweet.source_url)
    dict_tweets_info['text'].append(tweet.text)
    dict_tweets_info['lang'].append(tweet.lang)

    print('ATUAL DICIONARIO  : \n\n'+str(dict_tweets_info))
    return dict_tweets_info 

In [58]:
def export_infos_to_csv(dict_tweets_info):
    logging('\n\nfunction>>>>>exporting_infos_to_csv')
    
    now = datetime.now()
    current_directory = os.getcwd()  
    timestamp = now.strftime("%d/%m/%Y, %H:%M:%S").replace("/","-").replace(':',"-").replace(',','--').replace(" ","")

    new_path = current_directory+'\\arquivos_bot\\dados_exportados\\dados_'+timestamp+'.csv'
    logging('log path: '+str(new_path))
    
    logging('dict_tweets_info '+str(dict_tweets_info))
    
    df = pd.DataFrame.from_dict(dict_tweets_info) # turning into data frame
    df.to_csv(new_path,index=False)

In [59]:
def logging(text_to_log=""):
    
    # converte parâmetro que ele quer escrever no log em string, caso tenha sido enviado em outro formato
    text_to_log = str(text_to_log)
    
    # prepara data e timestamp pra dar append no arquivo de log (ou escrever um novo) + timestamp no conteúdo do arquivo
    now = datetime.now()
    date = now.strftime("%d/%m/%Y").replace("/","-")
    timestamp = now.strftime("%d/%m/%Y, %H:%M:%S")
    
    # busca diretório onde o robô está rodando e formata o path pro log do dia corrente
    current_directory = os.getcwd()  
    log_path = current_directory+'\\arquivos_bot\\logs\\log_'+date+'.txt'

    # se arquivo do dia já existir, só escreve nele o timestamp + conteúdo do parámetro
    # caso contrário, cria o arquivo de log daquele dia 
    with open(log_path, 'a+',encoding="utf-8") as log_file:
        log_file.write(timestamp+ ' - ' + text_to_log+'\n')
    
    # além de escrever no arquivo, printa no jupyter
    print(timestamp+ ' - ' + text_to_log)

In [60]:
def main():
    
    # cria pastas que o robô usa caso não existam 
    current_directory = os.getcwd()
    if not os.path.exists(current_directory+'\\arquivos_bot\\logs'):
        pymsgbox.alert(text='Criando pasta de logs', title='Preparando bot', button='OK',timeout=4500)
        os.makedirs(current_directory+'\\arquivos_bot\\logs')
        logging("Preparando ambiente")
        logging("Criando pasta de logs")
    
    if not os.path.exists(current_directory+'\\arquivos_bot\\controle'):
        pymsgbox.alert(text='Criando pasta de controle', tistle='Preparando bot', button='OK',timeout=4500)
        os.makedirs(current_directory+'\\arquivos_bot\\controle')
        logging("Criando pasta de controle")

    if not os.path.exists(current_directory+'\\arquivos_bot\\dados_exportados'):
        pymsgbox.alert(text='Criando pasta de dados exportados', title='Preparando bot', button='OK',timeout=4500)
        os.makedirs(current_directory+'\\arquivos_bot\\dados_exportados')
        logging("Criando pasta de dados exportados")

    pymsgbox.alert('Pastas necessárias para o robô conferidas, iniciando o bot', 'Starting bot',timeout=5000)
    logging(" --------- INICIANDO ROBÔ ---------")
    today = date.today().strftime("%B %d, %Y")
    print('today is:  '+today)
    
    credential =  {
                    "api_key" : credentials.API_KEY,
                    "api_secret" : credentials.API_SECRET,
                    "bearer_token" : credentials.BEARER_TOKEN,
                    "access_token" : credentials.ACCESS_TOKEN,
                    "access_token_secret" : credentials.ACCESS_TOKEN_SECRET
                    }
    
    api = authenticating(credential)
    
    ################## EM ALGUM MOMENTO PRECISO VERIFICAR SE A QUNATIDADE DE TWEETS NO DIA CORRENTE CHEGOU A 999, SE SIM 
    ################## PARAR DE EXECUTAR O BOT 
    dict_tweets_info = {
        
   #"created_at": [],
   #"place":      [],
   #"user":       [], 
   "id_str":     [],
   #"source_url": [],
    "text":       [],
    "lang":       []
                      }
    
    for tweet in tweepy.Cursor(api.search, q='zolpidem').items(2):
        
        print_and_retweet_tweet(api,tweet)
        logging("esperando 2 min pra continuar")
        time.sleep(60*2)                    # sleep 2 min, so we dont reach the limit 100 tweets per hour
        
        dict_tweets_info = append_tweet_infos_to_lists(tweet,dict_tweets_info)
        
        export_infos_to_csv(dict_tweets_info)

In [ ]:
def translate_special_text_to_ascii(original_text):
    translated_text = ''

    for character in original_text:
        if ord(character) >= 128:
            translated_text = translated_text + '"Chr(' + str(ord(character)) + ')"'
        else:
            translated_text = translated_text + character

    return translated_text

In [61]:
import import_ipynb
import credentials
import tweepy
import time
from datetime import date, datetime 
import os
import pymsgbox 
import pandas as pd

main()

02/01/2021, 18:01:05 -  --------- INICIANDO ROBÔ ---------
today is:  January 02, 2021
02/01/2021, 18:01:05 - 

function>>>>>authenticating
02/01/2021, 18:01:06 - 

function>>>>>print_and_retweet_tweet
02/01/2021, 18:01:06 - INFORMACOES:------------------------------




02/01/2021, 18:01:06 - tweet objectStatus(_api=<tweepy.api.API object at 0x000001C78B815B20>, _json={'created_at': 'Sat Jan 02 20:51:13 +0000 2021', 'id': 1345472738224431105, 'id_str': '1345472738224431105', 'text': 'alguém arruma uma receita de zolpidem pra mim por favor', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'pt', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2546729196, 'id_

02/01/2021, 18:01:06 - :::::::: retweeted 
02/01/2021, 18:01:06 - esperando 2 min pra continuar
02/01/2021, 18:03:06 - 

function>>>>>append_tweet_infos_to_lists
ATUAL DICIONARIO  : 

{'id_str': ['1345472738224431105'], 'text': ['alguém arruma uma receita de zolpidem pra mim por favor'], 'lang': ['pt']}
02/01/2021, 18:03:06 - 

function>>>>>exporting_infos_to_csv
02/01/2021, 18:03:06 - log path ao bater os 10 tweets: C:\Users\gabri\Documents\ZolpiBot\arquivos_bot\dados_exportados\dados_02-01-2021--18-03-06.csv
02/01/2021, 18:03:06 - dict_tweets_info {'id_str': ['1345472738224431105'], 'text': ['alguém arruma uma receita de zolpidem pra mim por favor'], 'lang': ['pt']}
02/01/2021, 18:03:06 - 

function>>>>>print_and_retweet_tweet
02/01/2021, 18:03:07 - INFORMACOES:------------------------------




02/01/2021, 18:03:07 - tweet objectStatus(_api=<tweepy.api.API object at 0x000001C78B815B20>, _json={'created_at': 'Sat Jan 02 20:46:04 +0000 2021', 'id': 1345471442339696640, 'id_str': '1345

02/01/2021, 18:03:07 - place: None
02/01/2021, 18:03:07 - source_url: http://twitter.com/download/iphone
02/01/2021, 18:03:07 - lang: pt
02/01/2021, 18:03:07 - ---------------------------------------------




02/01/2021, 18:03:07 - retweeting now::::::::
02/01/2021, 18:03:07 - :::::::: retweeted 
02/01/2021, 18:03:07 - esperando 2 min pra continuar
02/01/2021, 18:05:07 - 

function>>>>>append_tweet_infos_to_lists
ATUAL DICIONARIO  : 

{'id_str': ['1345472738224431105', '1345471442339696640'], 'text': ['alguém arruma uma receita de zolpidem pra mim por favor', 'Zolpidem é pra uso recreativo'], 'lang': ['pt', 'pt']}
02/01/2021, 18:05:07 - 

function>>>>>exporting_infos_to_csv
02/01/2021, 18:05:07 - log path ao bater os 10 tweets: C:\Users\gabri\Documents\ZolpiBot\arquivos_bot\dados_exportados\dados_02-01-2021--18-05-07.csv
02/01/2021, 18:05:07 - dict_tweets_info {'id_str': ['1345472738224431105', '1345471442339696640'], 'text': ['alguém arruma uma receita de zolpidem pra mim por favor',